## Exercise 2

*In this Exercise we work on the Simlex-999 dataset which gives the similarity score between two words and we compare it with the pretrained 300-dimensional word2vec embeddings from Google using gensim and find the similarity scores from this embeddings and solving the euclidean distance between two words and compare the similarity using the p test* 

In [1]:
import numpy as np
import os


*We build the reader function for reading in from .vecs files*

In [10]:


def dataset_reader(path):
    
    words1 = []
    words2 = []
    POSs = []
    SimLex999s = []
    concs_w1 = []
    concs_w2 = []
    concQs = []
    Assoc_USFs = []
    SimAssocs333 = []
    SD_SimLexs = []
    rows = []
    
    with open(path, "r") as f:
            next(f)
            for line in f:
                word1, word2, POS, SimLex999, conc_w1, conc_w2, concQ, Assoc_USF, SimAssoc333, SD_SimLex = line.split('\t')
                words1.append(word1)
                words2.append(word2)
                POSs.append(POS)
                SimLex999 = np.array(SimLex999.split(), dtype='float')
                SimLex999s.append(SimLex999)
                conc_w1 = np.array(conc_w1.split(), dtype='float')
                concs_w1.append(conc_w1)
                conc_w2 = np.array(conc_w2.split(), dtype='float')
                concs_w2.append(conc_w2)
                concQ = np.array(concQ.split(), dtype='float')
                concQs.append(concQ)
                Assoc_USF = np.array(Assoc_USF.split(), dtype='float')
                Assoc_USFs.append(Assoc_USF)
                SimAssoc333 = np.array(SimAssoc333.split(), dtype='float')
                SimAssocs333.append(SimAssoc333)
                SD_SimLex = np.array(SD_SimLex.split(), dtype='float')
                SD_SimLexs.append(SD_SimLex)
            
            strings = np.concatenate((np.array(words1,dtype=str).reshape(len(words1),1),np.array(words2,dtype=str).reshape(len(words1),1),np.array(POSs,dtype=str).reshape(len(words1),1)),axis=1)
            numbers = np.concatenate((np.array(SimLex999s).reshape(len(words1),1),np.array(concs_w1,dtype=float).reshape(len(words1),1) \
                                         ,np.array(concs_w2).reshape(len(words1),1),np.array(concQs).reshape(len(words1),1) \
                                         ,np.array(Assoc_USFs).reshape(len(words1),1),np.array(SimAssocs333).reshape(len(words1),1) \
                                         ,np.array(SD_SimLexs).reshape(len(words1),1),np.array(SimLex999s).reshape(len(words1),1))
                                         ,axis=1,dtype=float)
    return np.concatenate((strings,numbers),dtype=object,axis=1)

            
        

In [11]:
data = dataset_reader(r'C:\Users\adity\DL4NLP\hw02\SimLex-999\SimLex-999.txt')

In [12]:
print("Simlex-999-based similarities for (happy,cheerful) is : {}".format(data[np.where((data[:,0]=='happy') & (data[:,1]=='cheerful')),3]))
print("Simlex-999-based similarities for (happy,young) is : {}".format(data[np.where((data[:,0]=='happy') & (data[:,1]=='young')),3]))
print("Simlex-999-based similarities for (happy,angry) is : {}".format(data[np.where((data[:,0]=='happy') & (data[:,1]=='angry')),3]))

Simlex-999-based similarities for (happy,cheerful) is : [[9.55]]
Simlex-999-based similarities for (happy,young) is : [[2.0]]
Simlex-999-based similarities for (happy,angry) is : [[1.28]]


In [5]:
from gensim.models import KeyedVectors

wv_from_bin = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

*Function to build the euclidean distance betwen each word pair*

In [6]:
def dist_bw_each_pair(vec1,vec2):
    dist = np.empty((1,1),dtype=float)
    #print(vec1.shape)
    #for i in range(len(vec1)):
        #dist = dist + (vec1[i] - vec2[i])**2
    dist = np.sum((vec1 - vec2)**2)
    #print(dist)
    
    return np.sqrt(dist)

*Function to build the euclidean distance between all pairs*

In [7]:

def dist_bw_all_pairs(data):

    dist_words = np.empty((999,1),dtype=float)
    for i in  range(data.shape[0]):
    
        dist_words[i,0] = dist_bw_each_pair(wv_from_bin[data[i,0]],wv_from_bin[data[i,1]])
        
    return np.concatenate((np.array(data[:,0]).reshape(len(data),1),np.array(data[:,1]).reshape(len(data),1),np.array(dist_words)),axis=1,dtype=object)

In [8]:
distances = dist_bw_all_pairs(data)

In [9]:
print("word2vec-based distances between the pairs (happy, cheerful) is : {}" \
      .format(distances[np.where((distances[:,0]=='happy')& (distances[:,1]=='cheerful')),2]))
print("word2vec-based distances between the pairs (happy, young) is : {}" \
      .format(distances[np.where((distances[:,0]=='happy')& (distances[:,1]=='young')),2]))
print("word2vec-based distances between the pairs (happy, angry) is : {}" \
      .format(distances[np.where((distances[:,0]=='happy')& (distances[:,1]=='angry')),2]))

word2vec-based distances between the pairs (happy, cheerful) is : [[2.9259684085845947]]
word2vec-based distances between the pairs (happy, young) is : [[2.9159836769104004]]
word2vec-based distances between the pairs (happy, angry) is : [[2.874587059020996]]


*We do pearson coefficient analysis on the two similarity scores of simlex-999 and the similarity score we derived above*

In [550]:
from scipy import stats
stats.pearsonr(distances[:,2],data[:,3])

(-0.3474894638818695, 9.970478185540422e-30)

*From the above scores we can see that there is a moderate relationship between the two scores*